In [1]:
import pandas as pd
import numpy as np
import os
import json
import spacy

In [4]:
def read_json_file(path):
    f = open(path)
    json_content = json.load(f)
    f.close()
    return json_content


training_data_directory = 'data/rumoureval-2019-training-data/twitter-english'
test_data_directory = 'data/rumoureval-2019-test-data/twitter-en-test-data'

training_data_reddit_directory_dev = 'data/rumoureval-2019-training-data/reddit-dev-data'
training_data_reddit_directory = 'data/rumoureval-2019-training-data/reddit-training-data'
test_data_reddit_directory = 'data/rumoureval-2019-test-data/reddit-test-data'

training_labels_json = 'data/rumoureval-2019-training-data/train-key.json'
training_labels_json_2 = 'data/rumoureval-2019-training-data/dev-key.json'
test_labels_json = 'data/final-eval-key.json'

training_labels_dict = read_json_file(training_labels_json)['subtaskaenglish']
training_labels_dict.update(read_json_file(training_labels_json_2)['subtaskaenglish'])
test_labels_dict = read_json_file(test_labels_json)['subtaskaenglish']

In [32]:
class Tweet:
    def __init__(self, text,favorite_count, id, hastags,
                 retweet_count, verified,  parent_id=None):
        self.id = id
        self.favorite_count = favorite_count
        self.text= text
        self.parent_id = parent_id
        self.user_metadata = None
        self.hastags=hastags
        self.retweet_count=retweet_count
        self.verified = verified,
        self.direct_replies = []



In [37]:
def read_tweets_dataset(dataset_dir_path):
    topic_directories = [f.path for f in os.scandir(dataset_dir_path) if f.is_dir()]

    tweets = []
    for topic_dir in topic_directories:



        tweets_paths = [f.path for f in os.scandir(topic_dir) if f.is_dir()]
        for tweet_dir in tweets_paths:
            source_tweet_path = [f.path for f in os.scandir(tweet_dir + '/source-tweet')][0]
            source_tweet_json = read_json_file(source_tweet_path)
            source_tweet_hastags = source_tweet_json['entities']['hashtags']
            hashtags_text_array = [h['text'] for h in source_tweet_hastags]
            tweet = Tweet(source_tweet_json['text'],id= source_tweet_json['id'],
                          hastags=hashtags_text_array,
                          verified=source_tweet_json['user']['verified'],
                          retweet_count=source_tweet_json['retweet_count'],
                          favorite_count=source_tweet_json['favorite_count'],
                          )




            reply_tweets = []
            reply_tweets_paths = [f.path for f in os.scandir(tweet_dir + '/replies')]
            for reply_tweet_path in reply_tweets_paths:
                reply_tweet_json = read_json_file(reply_tweet_path)
                reply_tweet_hastags = reply_tweet_json['entities']['hashtags']
                hashtags_text_array = [h['text'] for h in reply_tweet_hastags]
                reply_tweet = Tweet(reply_tweet_json['text'],
                                    id=reply_tweet_json['id'],
                                  hastags=hashtags_text_array,
                                    parent_id=reply_tweet_json['in_reply_to_status_id'],
                                  verified=reply_tweet_json['user']['verified'],
                                  retweet_count=reply_tweet_json['retweet_count'],
                                  favorite_count=reply_tweet_json['favorite_count'],
                                    )
                reply_tweets.append(reply_tweet)
            tweets.append({'source':tweet, 'replies':reply_tweets})


    return tweets

In [38]:
tweets = read_tweets_dataset(training_data_directory)
for tweet in tweets:
    source_id = tweet['source'].id
    for dabila in tweet['replies']:
        if dabila.parent_id == source_id:
            tweet['source'].direct_replies.append(dabila)
        for abudzaba in tweet['replies']:
            if abudzaba.id == dabila.parent_id:
                abudzaba.direct_replies.append(dabila)
print(tweets[0]['source'].direct_replies)

print(len(tweets[0]['replies']))

[<__main__.Tweet object at 0x00000274F19619C0>, <__main__.Tweet object at 0x00000274F19610C0>, <__main__.Tweet object at 0x00000274F1961180>]
5


In [48]:
print(tweets[1]['source'].id)
print(tweets[1]['source'].direct_replies[0].id)
print(tweets[1]['source'].direct_replies[0].direct_replies[0].text)

552785375161499649
552785851655393281


IndexError: list index out of range